# Toy Radix Lagrangean decomposition

In [1]:
REG_WEIGHT   = 1e-4
MAX_NONZERO  = 48*2

In [2]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from cobra.io import load_json_model
from six import iteritems
import numpy as np
import cobra

In [3]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')
mdl_ref = ijomc
crowding_bound0 = mdl_ref.metabolites.crowding._bound

In [4]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [5]:
df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

ex_rxns = [r for r in df_meas.ex_rxn.unique() if mdl_ref.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()

#N_CONDS = len(conds)
N_CONDS = 2

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])

if N_CONDS<=3:
    df_conds = df_conds[ df_conds.cond.isin(['glucose','acetate','succinate'][0:N_CONDS])]
else:
    df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

df_conds.loc[ (df_conds.cond=='acetate') & (df_conds.rxn=='EX_ac_e'), 'lb'] = -20

In [6]:
from dynamicme.decomposition import LagrangeSubmodel, LagrangeMaster, BendersMaster, BendersSubmodel, DecompModel
from dynamicme.optimize import Optimizer
from dynamicme.generate import copy_model

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7fbc0d37a050>)


In [7]:
from dynamicme.optimize import Variable, Constraint
import numpy as np

PREVENT_ZERO = True

radix = 2.
print('Radix:',radix)
#powers = np.arange(-3,4)
powers = [-1, 0, 1]
print('Powers:', powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print('Digits:', digits)

# Get the group ID from reference model
mu_id = 'BIOMASS_Ecoli_core_w_GAM'
mdl_ref = ijomc
crowding_ref = mdl_ref.metabolites.crowding
conds = df_conds.cond.unique()
psub_dict = {}
dsub_dict = {}
for cond in conds:
    mdl_ind = cond
    mdl = copy_model(ijomc, suffix='_%s'%mdl_ind)            
    opt = Optimizer(mdl)
    gap = opt.add_duality_gap_constraint(INF=1e3, inplace=True, index=mdl_ind)
    #----------------------------------------------------
    # Now, add min abs err
    #----------------------------------------------------
    for rxn in gap.reactions:
        rxn.objective_coefficient = 0.
    mu_meas = df_meas[ df_meas.substrate==mdl_ind].growth_rate_1_h.iloc[0]
    sp = Variable('sp_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sn = Variable('sn_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sp.objective_coefficient = 1.
    sn.objective_coefficient = 1.
    cons = Constraint('abs_err_%s'%mdl_ind)
    cons._constraint_sense = 'E'
    cons._bound = mu_meas
    gap.add_metabolites(cons)
    gap.add_reactions([sp,sn])
    # mu - mu_meas = sp-sn
    # mu -sp + sn = mu_meas
    # min sp + sn
    sp.add_metabolites({cons:-1.})
    sn.add_metabolites({cons:1.})
    rxn_mu = gap.reactions.get_by_id(mu_id+'_%s'%mdl_ind)
    rxn_mu.add_metabolites({cons:1.})
    #----------------------------------------------------
    
    dfi = df_conds[ df_conds.cond==cond]
    var_cons_dict = {}
    for rxn_ref in crowding_ref.reactions:    
        crowding_p = mdl.metabolites.get_by_id('crowding_%s'%mdl_ind)
        var_d = mdl.reactions.get_by_id('wa_%s'%crowding_p.id)
        rxn_p = mdl.reactions.get_by_id(rxn_ref.id+'_%s'%mdl_ind)
        cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]        
        a0 = rxn_p.metabolites[crowding_p]
        if var_cons_dict.has_key(rxn_ref.id):
            var_cons_dict[rxn_ref.id] += [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        else:
            var_cons_dict[rxn_ref.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        
    opt.to_radix(gap, var_cons_dict, radix, powers, digits=digits, prevent_zero=PREVENT_ZERO)    
    
    dsub = LagrangeSubmodel(gap, cond)
    dsub_dict[cond] = dsub
    
    psub= BendersSubmodel(gap, cond)
    psub_dict[cond] = psub
        
dmaster = LagrangeMaster(gap)
pmaster = BendersMaster(gap)

('Radix:', 2.0)
('Powers:', [-1, 0, 1])
('Digits:', [1.0])
Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:13 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05


In [ ]:
pmaster.add_submodels(psub_dict)
dmaster.add_submodels(dsub_dict)

In [ ]:
#%%time
import time

max_iter = 100
gaptol = 1e-3

pmaster.model.Params.OutputFlag =0
dmaster.model.Params.OutputFlag = 0

print_int = 10

UB = 1e15
LB = -1e15
delta = 1.

warm_start = False

print("%-15.10s%-15.10s%-15.10s%-15.10s%-15.10s%-15.10s" % ('Iter','Gap','UB','LB','Relgap (%)','Time (s)'))

#------------------------------------
# 1) Init
ny = len(pmaster._y0)
yopt = np.zeros(ny)
nu = len(dmaster._us)
uk = np.zeros(nu)
LB = -1e15
UB = 1e15
gap = 1e15
#------------------------------------
tic = time.time()
_iter = 0.
while _iter < max_iter:
    #------------------------------------
    # 2) Benders subproblems
    pobj_dict={}
    for sub_ind,sub in iteritems(pmaster.sub_dict):
        sub.update_obj(yopt)
        sub.model.optimize()
        zp = 1e15
        if sub.model.Status==GRB.OPTIMAL:
            zp = sub._weight * sub.model.ObjVal
            optcut = pmaster.make_optcut(sub)
            pmaster.model.addConstr(optcut)
        elif sub.model.Status==GRB.UNBOUNDED:
            feascut = pmaster.make_feascut(sub)
            pmaster.model.addConstr(feascut)
        pobj_dict[sub_ind] = zp
        # Add cut to Dual master:
        # tk <= zpk* + u*H*y
        dsub = dmaster.sub_dict[sub_ind]
        benders_cut = dmaster.make_benderscut(dsub, zp, yopt)
        dmaster.model.addConstr(benders_cut)
    fy = pmaster._fy
    UBk = sum(fy*yopt) + sum(pobj_dict.values())
    if UBk < UB:
        UB = UBk
    #------------------------------------
    # 3) Dual subproblems
    dobj_dict = {}
    for sub_ind,sub in iteritems(dmaster.sub_dict):
        sub.update_obj(uk)
        #sub.model.Params.OutputFlag = 1
        sub.model.Params.TimeLimit = 300.
        #if sub.model.Status==GRB.OPTIMAL:
        if warm_start:
            if sub.model.SolCount > 0:
                xk = [x.X for x in sub._xs]
                yk = [y.X for y in sub._ys]
                sub.optimize(xk,yk)
        else:
            sub.model.optimize()
        
        if sub.model.SolCount > 0:
            yk = [y.X for y in sub._ys]
            xk = [x.X for x in sub._xs]
            cut = dmaster.make_optcut(sub,xk,yk)
            dmaster.model.addConstr(cut)
        else:
            #print("%-15.10d%-20.15g%-20.15g%-20.15g" % (_iter, ))
            print("%-15.10d%s" % (_iter, "No solution in submodel!"))
        
        sub_obj = sub.model.ObjVal
        dobj_dict[sub_ind] = sub_obj                
    #------------------------------------
    # 4) Dual Master
    dmaster.model.update()
    dmaster.model.Params.TimeLimit = 300.
    dmaster.model.optimize()
    uk = np.array([u.X for u in dmaster._us])
    #------------------------------------
    # 5) Benders Master
    pmaster.model.update()
    pmaster.model.optimize()
    yopt = np.array([y.X for y in pmaster._ys])    
    LB = pmaster._z.X
    #------------------------------------          
    # 6) CHECK GAP    
    gap = UB-LB
    relgap = gap/(UB+1e-10)
    if np.mod(_iter, print_int)==0:
        toc = time.time()-tic
        print("%-15.10s%-15.5g%-15.5g%-15.5g%-15.5g%-15.5g" % (_iter, gap, UB, LB, relgap*100,toc ))    
    if relgap < gaptol:
        print("Gap < Gaptol at iter %d"%_iter)
        break
    else:
        delta = max(delta/2., 1e-10)
        dmaster.update_obj(delta)
    _iter+=1

Iter           Gap            UB             LB             Relgap (%)     Time (s)       
0.0            4.1229         0.75184        -3.371         548.37         0.317          
10.0           2.2946         0.484          -1.8106        474.08         30.485         
20.0           2.2712         0.484          -1.7872        469.25         32.099         


# Does final yopt work?

## Are all yks the same?

In [ ]:
yids = [y.VarName for y in sub._ys]
key0 = dmaster.sub_dict.keys()[0]
sub0 = dmaster.sub_dict[key0]
yk0  = np.array([sub0.model.getVarByName(yid).X for yid in yids])

err_dict = {}
for sub_ind,sub in iteritems(master.sub_dict):
    yk = np.array([sub.model.getVarByName(yid).X for yid in yids])    
    err_dict[sub_ind] = sum(yk != yk0)

In [ ]:
err_dict

In [ ]:
for sub_ind,sub in iteritems(master.sub_dict):
    sol_sub = {x.VarName:x.X for x in sub.model.getVars()}
    yopt = [y.X for y in sub._ys]
    print('Number of non-zero binaries: %g' % sum(yopt))
    # Fitted parameters
    kfit_dict = {}
    for group_id, var_dict in iteritems(var_cons_dict):
        var = var_dict[0]
        cons = var_dict[1]
        a0  = var_dict[0][2]
        kfit = 0.
        for l,pwr in enumerate(powers):
            for k,digit in enumerate(digits):            
                yid = 'binary_%s%s%s' % (group_id,k,l)
                y   = sol_sub[yid]
                kfit += y*a0*radix**pwr*digit
        kfit_dict[group_id] = kfit

    kfit_changed = [(k,v, abs(v-a0)/a0) for k,v in iteritems(kfit_dict) if abs(v-a0)/a0>1e-6]
    print('Changed keffs: %d/%d' % (len(kfit_changed), len(var_cons_dict)))
    kfit_changed

    #----------------------------------------
    # Starting from basal model
    csrcs = df_conds.cond.unique()
    for csrc in csrcs:
        ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')    
        crowding = Constraint('crowding')
        crowding._bound = crowding_bound0
        crowding._constraint_sense = 'L'

        df_condi = df_conds[ df_conds.cond==csrc]    
        for i,row in df_condi.iterrows():
            rid = row['rxn']
            rxn = ijofit.reactions.get_by_id(rid)
            rxn.lower_bound = row['lb']
            rxn.upper_bound = row['ub']

        for rid,kfit in iteritems(kfit_dict):
            rxn = ijofit.reactions.get_by_id(rid)
            rxn.add_metabolites({crowding:kfit})

        ijofit.optimize()

        mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
        mu_fiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x

        # Get unfit
        for rxn in ijofit.metabolites.crowding.reactions:
            rxn._metabolites[crowding] = a0
        ijofit.optimize()
        mu_unfiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
        err0= 100*(mu_unfiti-mu_measi)/mu_measi
        err = 100*(mu_fiti - mu_measi)/mu_measi
        derr= 100*(abs(err)-abs(err0))/abs(err0)
        print('Cond=%s. mu_meas=%g. mu_sim=%g (unfit=%g, error=%.3g%%). Error=%.3g%% (%.3g%% change)' % (csrc, mu_measi, mu_fiti, mu_unfiti, err0, err, derr))
        for i,row in df_condi.iterrows():
            rid = row['rxn']
            rxn = ijofit.reactions.get_by_id(rid)        
            print('\t%s uptake=%g' % (rxn.id, rxn.x))